# Exercise 6.14

In [1]:
import pathlib
import subprocess
import numpy as np
import pandas as pd
from IPython.display import display, Math
from scipy import linalg as la
from scipy import stats
import TwoWayManova
from BonferroniSimultaneousConfidenceIntervals import BonferroniSimultaneousConfidenceIntervals
from TwoWayManovaEffectTests import TwoWayManovaEffectTests

In [2]:
def load_data():
    f = pathlib.WindowsPath(r'..\..\Data\Exercise6.14.xlsx')
    return pd.read_excel(f)

In [3]:
df = load_data()

## (a)

In [4]:
fit = TwoWayManova.TwoWayManova(df, factor1_col='Factor1', factor2_col='Factor2', var_cols=['x1', 'x2'])

In [5]:
fit.x_breakdown.display_obs_breakdown(['1.05cm','2.5cm','2.2cm','1.1cm','1.5cm','1.75cm'])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## (b)

In [6]:
fit.display_manova_table()

<IPython.core.display.Math object>

## (c)

In [7]:
TwoWayManovaEffectTests(fit, alpha=0.05).display_test_results()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## (d)

In [8]:
alpha = 0.05
BonferroniSimultaneousConfidenceIntervals(fit, alpha).display_two_way_manova_bonf()

Factor1


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Notice how almost every confidence interval contains 0. The only exception is the pairwise comparison of $\tau_{1,1} - \tau_{3,1}$, so there is a difference in means between level 1 and level 3 for Factor 1 variable 1. This is what's causing Factor 1 main effect to be significant. All difference confidence intervals contain 0 for Factor 2, which is consistent with there being no significant effects for Factor 2.

## MANOVA results from R

Using R we can get the same output as SAS. MANOVA in Python unforunately doesn't match up with everyone else.

In [9]:
result = subprocess.run([r'C:\Program Files\R\R-4.4.2\bin\Rscript', '--vanilla', r'..\..\r\chapter-6\Question-6-14.R'], capture_output=True, text=True)
print(result.stdout)